<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
import qiskit
import numpy as np
from qiskit.aqua.operators import PauliExpectation, CircuitSampler, ExpectationFactory
from qiskit.aqua.operators import CircuitStateFn, StateFn, ListOp

In [20]:
from typing import Union
import qiskit
import numpy as np
from qiskit.aqua.utils.backend_utils import is_aer_provider
from qiskit.aqua.operators import (PauliExpectation, CircuitSampler, ExpectationFactory,
                                   CircuitStateFn, StateFn, ListOp)

def sample_hamiltonian(hamiltonian: qiskit.aqua.operators.OperatorBase,
                       backend: Union[qiskit.providers.BaseBackend, qiskit.aqua.QuantumInstance],
                       ansatz: qiskit.QuantumCircuit):
    
    sampler = CircuitSampler(backend, param_qobj=is_aer_provider(backend.backend))
    expectation = ExpectationFactory.build(operator=hamiltonian,backend=backend)
    observable_meas = expectation.convert(StateFn(hamiltonian, is_measurement=True))
    
    ansatz_circuit_op = CircuitStateFn(wavefunction)
    
    expect_op = observable_meas.compose(ansatz_circuit_op).reduce()
    sampled_expect_op = sampler.convert(expect_op)
    
    return np.real(sampled_expect_op.eval())

In [43]:
from Ansatz import get_num_parameters, get_var_form
from qiskit.aqua.operators import I, X, Y, Z
from qiskit.aqua import QuantumInstance

n_qubits = 2
n_params = get_num_parameters(n_qubits)
params = [0.]*n_params
wavefunction = get_var_form(params, n_qubits)

hamiltonian = 1/2*((Z^Z) + (Z^I) + (X^X))

backend = QuantumInstance(backend=qiskit.BasicAer.get_backend('qasm_simulator'))
sample_hamiltonian(hamiltonian, backend, wavefunction)

1.9921875

In [25]:
# Variational Form

backend = qiskit.BasicAer.get_backend('qasm_simulator')
sampler = CircuitSampler(backend=backend)

expectation = ExpectationFactory.build(operator=hamiltonian,backend=backend)
observable_meas = expectation.convert(StateFn(hamiltonian, is_measurement=True))


In [13]:
print(observable_meas)

ComposedOp([
  OperatorMeasurement(ZZ),
       ┌─────┐┌───┐
  q_0: ┤ SDG ├┤ H ├
       ├─────┤├───┤
  q_1: ┤ SDG ├┤ H ├
       └─────┘└───┘
])


In [11]:
ansatz_circuit_op = CircuitStateFn(wavefunction)
expect_op = observable_meas.compose(ansatz_circuit_op).reduce()
sampled_expect_op = sampler.convert(expect_op)
means = np.real(sampled_expect_op.eval())

In [12]:
means

1.0